# Introduction: Intermediate Time Series methods with Plotly

In this notebook, we will move beyond the basic plots in the `plotly-time-series` notebook and make plots with range sliders, update menus, and even animations. Although still not using the full capabilities of plotly, making these plots will show us how to take advantage of this powerful library to create effective visualizations. 

In [3]:
# Standard data science libraries
import pandas as pd
import numpy as np

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('bmh')

from collections import defaultdict

Authenticate with plotly in the below cell.

In [4]:
## Replace with your credentials.

import plotly
# plotly.tools.set_credentials_file(username='########', api_key='******')

These are the standard plotly imports. We set up the notebook to run offline which means our plots are not uploaded to the plot web interface.

In [5]:
pip install chart-studio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Plot imports
import chart_studio.plotly as py
import plotly.graph_objs as go

## Offline mode
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

## Data Series

As before, we are using real-world building energy, steam, and static pressure measurements. The data is in a dataframe with a _multi-index_ on the columns. This means we have to use _multi-dimensional_ indexing to select one column.

In [7]:
# Read in data and convert index to a datetime
df = pd.read_csv('C:/Users/saite/OneDrive/Desktop/data/building_one.csv', 
                 header=[0, 1], index_col=0, )
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df.head()

type                StaticPressure                 Energy      Steam  \
sensor                           1         2            3          4   
measured_at                                                            
2017-12-14 11:00:00       0.806723  1.500570  6035.214040  13.164377   
2017-12-14 11:15:00       0.789395  1.490740  6182.405506  13.003065   
2017-12-14 11:30:00       0.792908  1.473761  6035.187942  12.617836   
2017-12-14 11:45:00       0.790165  1.485213  6035.192571  12.419816   
2017-12-14 12:00:00       0.786861  1.482015  6035.198581  12.975920   

type                            
sensor                       5  
measured_at                     
2017-12-14 11:00:00  10.367290  
2017-12-14 11:15:00   9.801097  
2017-12-14 11:30:00   9.794436  
2017-12-14 11:45:00   9.597848  
2017-12-14 12:00:00  10.041990

In [8]:
pip install cufflinks

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import cufflinks as cf
import pandas as pd
from plotly.offline import iplot, init_notebook_mode

# Initialize plotly in offline mode
init_notebook_mode(connected=True)
cf.go_offline()

# Sample data
data = {'Date': pd.date_range(start='2018-01-01', end='2018-01-15', freq='D'),
        'Steam': [100, 110, 115, 120, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180]}
df = pd.DataFrame(data)

# Plotting
df.iplot(x='Date', y='Steam', xTitle='Date', yTitle='Steam (Mlbs/hr)', 
         theme='white', title="Steam Plot",
         layout=dict(xaxis=dict(range=[pd.Timestamp(2018, 1, 1), pd.Timestamp(2018, 1, 15)])))


Next we create several data series, spanning both the entire range of data, and subsetting out one week.

In [10]:
print(df.columns)

Index(['Date', 'Steam'], dtype='object')


In [11]:
print(df.index)

RangeIndex(start=0, stop=15, step=1)


For example, if the DataFrame has MultiIndex columns, you should use:

In [48]:
import pandas as pd

# Assuming df is your DataFrame with a MultiIndex for columns
energy_series = df.loc[:, ('Energy', '3')].copy()
steam_series = df.loc[:, ('Steam', '4')].copy()
pressure_series = df.loc[:, ('StaticPressure', '2')].copy()


In [13]:
import plotly.graph_objects as go

In [14]:
# Sample data for demonstration
dates = pd.date_range(start='2024-01-01', periods=100, freq='D')
data = {
    ('Energy', '3'): abs(np.random.normal(loc=50, scale=10, size=100)),
    ('Steam', '4'): abs(np.random.normal(loc=100, scale=15, size=100)),
    ('StaticPressure', '2'): abs(np.random.normal(loc=200, scale=20, size=100)),
}
df = pd.DataFrame(data, index=dates)


In [15]:
energy_series = df[('Energy', '3')].copy()
steam_series = df[('Steam', '4')].copy()
pressure_series = df[('StaticPressure', '2')].copy()

In [16]:
# Create a figure
fig = go.Figure()

# Add traces for each series
fig.add_trace(go.Scatter(x=energy_series.index, y=energy_series.values, mode='lines', name='Energy'))
fig.add_trace(go.Scatter(x=steam_series.index, y=steam_series.values, mode='lines', name='Steam'))
fig.add_trace(go.Scatter(x=pressure_series.index, y=pressure_series.values, mode='lines', name='Static Pressure'))

# Update layout
fig.update_layout(
    title='Time Series Data',
    xaxis_title='Date',
    yaxis_title='Value',
    template='plotly_dark'
)

# Show the plot
fig.show()


In [17]:
pip install RangeIndex

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement RangeIndex (from versions: none)
ERROR: No matching distribution found for RangeIndex

[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
if hasattr(df.index, 'isocalendar'):
    df_short = df[df.index.isocalendar().week == 8].copy()
else:
    df_short = df[df.index.week == 8].copy()


# Time Scale and Time Window Selection with Range Select and Range Slider

One way to add interactivity to our time-series plots and let users dig into the data is by adjusting the time scale and the time window. We can do this using a `rangeselect` and `rangslider` respectively. 

In [19]:
# Create a layout with a rangeselector and rangeslider on the xaxis
layout = go.Layout(
    height=600,
    width=900,
    font=dict(size=16),
    title='Energy Plot with Range Selection',
    xaxis=dict(
        title='Date',
        # Range selector with buttons
        rangeselector=dict(
            font=dict(size=12),
            # Buttons for selecting time scale
            buttons=list([
                # 1 month
                dict(count=1, label='1m', step='month', stepmode='backward'),
                # 1 week
                dict(count=7, label='1w', step='day', stepmode='backward'),
                # 1 day
                dict(count=1, label='1d', step='day', stepmode='backward'),
                # 12 hours
                dict(count=12, label='12h', step='hour', stepmode='backward'),
                # 4 hours
                dict(count=4, label='4h', step='hour', stepmode='backward'),
                # Entire scale
                dict(step='all')
            ])),
        # Sliding for selecting time window
        rangeslider=dict(visible=True),
        # Type of xaxis
        type='date'),
    # yaxis is unchanged
    yaxis=dict(title='Energy (kWh)'))

In [20]:
# Create the same data object
energy_data = go.Scatter(x=energy_series.index,
                        y=energy_series.values,
                        line=go.scatter.Line(color='red', width = 0.6),
                        opacity=0.8,
                        name='energy',
                        text=[f'Energy: {x:.0f} kWh' for x in energy_series.values])

# Create the figure and display
fig = go.Figure(data=[energy_data], layout=layout)
iplot(fig)

## Time Select with Double Y-Axis

In [21]:
steam_data = go.Scatter(x=steam_series.index,
                        y=steam_series.values,
                        line=dict(color='blue', width=0.8),
                        opacity=0.8,
                        name='Steam',
                        yaxis='y2',
                        text=[f'Steam: {x:.1f} Mlbs/hr' for x in steam_series.values])

In [22]:
# Create a layout with interactive elements and two yaxes
layout = go.Layout(
    height=600,
    width=900,
    font=dict(size=16),
    title='Energy Plot with Range Selection',
    xaxis=dict(
        title='Date',
        # Range selector with buttons
        rangeselector=dict(
            font=dict(size=12),
            # Buttons for selecting time scale
            buttons=list([
                # 1 month
                dict(count=1, label='1m', step='month', stepmode='backward'),
                # 1 week
                dict(count=7, label='1w', step='day', stepmode='backward'),
                # 1 day
                dict(count=1, label='1d', step='day', stepmode='backward'),
                # 12 hours
                dict(count=12, label='12h', step='hour', stepmode='backward'),
                # 4 hours
                dict(count=4, label='4h', step='hour', stepmode='backward'),
                # Entire scale
                dict(step='all')
            ])),
        # Sliding for selecting time window
        rangeslider=dict(visible=True),
        # Type of xaxis
        type='date'),
    yaxis=dict(title='Energy (kWh)', color='red'),
    # Add a second yaxis to the right of the plot
    yaxis2=dict(
        title='Steam (Mlbs/hr)', color='blue', overlaying='y', side='right'))

fig = go.Figure(data=[energy_data, steam_data], layout=layout)
iplot(fig)

# Update Menu

Next, we will add a dropdown menu to the plot allowing users to select the steam series they want to view. This is an `update` because it changes the data shown. We just need to create an `updatemenus` object that specify the actions to take when the button is selected in the `args` parameter. In this case, we change the data by setting the `visible` parameters and we change the title with the `title` parameter.

In [23]:
updatemenus = list([
    dict(
        active=2,
        buttons=list([
            dict(
                label='Sensor 4',
                method='update',
                args=[{
                    'visible': [True, False]
                }, {
                    'title': 'Sensor 4'
                }]),
            dict(
                label='Sensor 5',
                method='update',
                args=[{
                    'visible': [False, True]
                }, {
                    'title': 'Sensor 5'
                }]),
            dict(
                label='Both',
                method='update',
                args=[{
                    'visible': [True, True]
                }, {
                    'title': 'Sensor Sensors'
                }])
        ]),
    )
])

Once we make the `updatemenus`, we pass in to the `layout`.

In [24]:
layout = go.Layout(
    height=800, width=1000, title='Steam Sensors', updatemenus=updatemenus)

Now we need to make our data. We will use two different steam series over the course of one week.

In [26]:


dates = pd.date_range(start='2024-01-01', periods=100, freq='D')
steam_series_four = pd.Series(abs(np.random.normal(loc=100, scale=10, size=100)), index=dates)
steam_series_five = pd.Series(abs(np.random.normal(loc=110, scale=15, size=100)), index=dates)

# Create Scatter traces
steam_data_four = go.Scatter(
    x=steam_series_four.index,
    y=steam_series_four.values,
    line=dict(color='blue', width=1.1),
    opacity=0.8,
    name='Steam: Sensor 4',
    text=[f'Sensor 4: {x:.1f} Mlbs/hr' for x in steam_series_four.values],
    mode='lines+markers',  # Show lines and markers
    marker=dict(size=5)    # Marker size
)

steam_data_five = go.Scatter(
    x=steam_series_five.index,
    y=steam_series_five.values,
    line=dict(color='orange', width=1.1),
    opacity=0.8,
    name='Steam: Sensor 5',
    text=[f'Sensor 5: {x:.1f} Mlbs/hr' for x in steam_series_five.values],
    mode='lines+markers',  # Show lines and markers
    marker=dict(size=5)    # Marker size
)

# Create a figure and add traces
fig = go.Figure()

fig.add_trace(steam_data_four)
fig.add_trace(steam_data_five)

# Update layout
fig.update_layout(
    title='Steam Data from Sensors 4 and 5',
    xaxis_title='Date',
    yaxis_title='Mlbs/hr',
    template='plotly_dark',
    hovermode='x unified'  # Show hover information for all traces at the same x position
)

# Show the plot
fig.show()


Finally, we pass in the layout with the `updatemenus` to the figure.

In [27]:
fig = go.Figure(data=[steam_data_four, steam_data_five], layout=layout)
iplot(fig)

## Update Menu with Annotations

Now we will add in the annotations when the user selects a sensor. This is simply a matter of changing the visible annotations when a different sensor is selected using the `annotations` parameter.

First, we format all of the data. We are using weekly series and adding in annotations that show the maximum value on each day.

In [28]:
def find_daily_maxes(x):
    """Return maximum measurement on each day and when it occurred in a dataframe"""
    x = x.copy().to_frame()
    x['day'] = x.index.day
    result =pd.concat([x.groupby('day').max(), 
                      x.groupby('day').idxmax()], axis = 1).iloc[:, [0, 1]]
    result.columns = ['value', 'date']
    return result.set_index('date')

four_highs = find_daily_maxes(steam_series_four)
five_highs = find_daily_maxes(steam_series_five)

from datetime import datetime

def format_time(dt):
    if pd.isnull(dt):
        return "NaT"
    else:
        return datetime.strftime(dt, "%a <br> %H:%M %p")
    
four_highs

,value
date,
2024-04-01,112.561486
2024-02-02,106.490883
2024-04-03,106.188525
2024-03-04,123.965287
2024-01-05,107.782191
2024-02-06,110.546007
2024-03-07,110.767742
2024-04-08,118.108332
2024-04-09,108.559179


The annotations are made using a list comprehension.

In [29]:
four_annotations = [
    dict(
        x=date,
        y=value[0],
        xref='x',
        yref='y',
        font=dict(color='blue'),
        text=f'{format_time(date)}<br> {value[0]:.1f} Mlbs/hr')
    for date, value in zip(four_highs.index, four_highs.values)
]

five_annotations = [
    dict(
        x=date,
        y=value[0],
        xref='x',
        yref='y',
        font=dict(color='orange'),
        text=f'{format_time(date)}<br> {value[0]:.1f} Mlbs/hr')
    for date, value in zip(five_highs.index, five_highs.values)
]

four_annotations[:2]

[{'x': Timestamp('2024-04-01 00:00:00'),
  'y': 112.56148554591458,
  'xref': 'x',
  'yref': 'y',
  'font': {'color': 'blue'},
  'text': 'Mon <br> 00:00 AM<br> 112.6 Mlbs/hr'},
 {'x': Timestamp('2024-02-02 00:00:00'),
  'y': 106.4908831927943,
  'xref': 'x',
  'yref': 'y',
  'font': {'color': 'blue'},
  'text': 'Fri <br> 00:00 AM<br> 106.5 Mlbs/hr'}]

In the `updatemenus` is where we specify the `annotations` that appear when a button is selected. We will only show annotations when an individual sensor is selected.

In [30]:
updatemenus = list([
    dict(
        active=2,
        buttons=list([
            dict(
                label='Sensor 4',
                method='update',
                args=[{
                    'visible': [True, False]
                }, {
                    'title': 'Sensor 4',
                    'annotations': four_annotations
                }]),
            dict(
                label='Sensor 5',
                method='update',
                args=[{
                    'visible': [False, True]
                }, {
                    'title': 'Sensor 5',
                    'annotations': five_annotations
                }]),
            dict(
                label='Both',
                method='update',
                args=[{
                    'visible': [True, True]
                }, {
                    'title': 'Sensor Sensors'
                }])
        ]),
    )
])

layout = go.Layout(
    height=600,
    width=800,
    title='Steam Sensors',
    xaxis=dict(tickformat='%a %b %d'),
    updatemenus=updatemenus)

In [31]:
fig = go.Figure(data = [steam_data_four, steam_data_five], 
                layout=layout)

iplot(fig)

# Animations

Finally, we will work on producing animated plots. This allows us to see how a figure may change over time.

In [32]:
spring = [3, 4, 5]
summer = [6, 7, 8]
fall = [9, 10, 11]
winter = [12, 1, 2]

int_to_days = {0: 'Mon', 1: 'Tues', 2: 'Wed', 3: 'Thurs',
               4: 'Fri', 5: 'Sat', 6: 'Sun'}

color_mapping = {'spring': 'green', 'summer': 'orange', 'fall': 'brown', 'winter':' blue'}

In [33]:
def format_season(season_months, season_name, indexer = ('Energy', '3'), name = 'Energy', units = 'kWh'):
    """Format data for one season"""
    data = df[df.index.month.isin(season_months)].copy()
    data_x = defaultdict(list)
    data_y = defaultdict(list)
    
    color = color_mapping[season_name]
    
    for (time, dow), grouped in data.groupby([data.index.time, data.index.dayofweek]):
        x = pd.datetime(2018, 3, 5 + dow, 0 + time.hour, 0 + time.minute)
        y = grouped[indexer].mean()
        data_x[dow].append(x)
        data_y[dow].append(y)
    
    data_obj = []
    for dow, x in data_x.items():
        y = data_y[dow]
        text = [f'{season_name} <br> {name}: {m:.2f} {units}' for m in y]
        data_obj.append(go.Scatter(x = x, y = y, text = text,
                                   hoverinfo='text',
                                   line = dict(color=color, width=0.75),
                                   name = season_name))
    
    return data_obj

In [35]:
def format_season(data, season):
    # Example processing based on the season
    return [{'name': f'{season.capitalize()} Data', 'data': data}]
winter_data_source = format_season(winter, 'winter')
def format_season(data, season):
    return [{'name': f'{season.capitalize()} Data', 'data': data}]

# Example winter data
winter = [1, 2, 3, 4, 5]

# Process data
winter_data_source = format_season(winter, 'winter')

# Accessing the 'name' key
print(winter_data_source[0]['name'])  # Output: 'Winter Data'


Winter Data


In [36]:
summer_data_source = format_season(summer, 'summer')
fall_data_source = format_season(fall, 'fall')
spring_data_source = format_season(spring, 'spring')

In [38]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Example data for each season (replace with your actual data)
dates = pd.date_range(start='2024-01-01', periods=30, freq='D')

summer_data_source = [go.Scatter(
    x=dates,
    y=abs(np.random.normal(loc=75, scale=10, size=30)),
    line=dict(color='orange', width=1.1),
    opacity=0.8,
    name='Summer'
)]

winter_data_source = [go.Scatter(
    x=dates,
    y=abs(np.random.normal(loc=50, scale=10, size=30)),
    line=dict(color='blue', width=1.1),
    opacity=0.8,
    name='Winter'
)]

fall_data_source = [go.Scatter(
    x=dates,
    y=abs(np.random.normal(loc=60, scale=10, size=30)),
    line=dict(color='green', width=1.1),
    opacity=0.8,
    name='Fall'
)]

spring_data_source = [go.Scatter(
    x=dates,
    y=abs(np.random.normal(loc=70, scale=10, size=30)),
    line=dict(color='red', width=1.1),
    opacity=0.8,
    name='Spring'
)]

# Define layout
layout = go.Layout(
    title='Seasonal Average Energy Use',
    xaxis=dict(nticks=7, tickformat='%A'),
    yaxis=dict(title='Energy Use (Mlbs/hr)')
)

# Create figure and plot
fig = go.Figure(
    data=summer_data_source + winter_data_source + fall_data_source + spring_data_source,
    layout=layout
)

fig.show()  # Use fig.show() for environments outside Jupyter notebooks


In [42]:

# Create a DatetimeIndex for demonstration
dates = pd.date_range(start='2024-03-01', end='2024-03-31', freq='H')
data = pd.DataFrame({
    'Energy': np.random.normal(loc=70, scale=10, size=len(dates))
}, index=dates)

# Add hour and minute columns
data['hours'] = data.index.hour
data['minutes'] = data.index.minute

# Group by time of day and day of the week, and compute the mean
weekly_data = data.groupby([data.index.time, data.index.dayofweek]).mean().reset_index()

# Rename columns
weekly_data.columns = ['Time', 'Day of Week', 'Energy', 'hours', 'minutes']

# Display the result
print(weekly_data.head())


       Time  Day of Week     Energy  hours  minutes
0  00:00:00            0  71.003812    0.0      0.0
1  00:00:00            1  58.812928    0.0      0.0
2  00:00:00            2  70.263130    0.0      0.0
3  00:00:00            3  67.930563    0.0      0.0
4  00:00:00            4  64.163868    0.0      0.0


In [46]:

from datetime import datetime, timedelta

# Example spring_weekly DataFrame
# Replace this with your actual DataFrame
dates = pd.date_range(start='2024-03-01', periods=30, freq='H')
data = pd.DataFrame({
    'Energy': np.random.normal(loc=70, scale=10, size=len(dates))
}, index=dates)
data['hours'] = data.index.hour
data['minutes'] = data.index.minute

weekly_data = data.groupby([data.index.time, data.index.dayofweek]).mean().reset_index()
weekly_data.columns = ['Time', 'Day of Week', 'Energy', 'hours', 'minutes']

# Creating Scatter traces
traces = []

base_date = datetime(2018, 3, 5)  # Base date for reference

for _, row in weekly_data.iterrows():
    # Calculate the actual date by adding days to base_date
    day_offset = row['Day of Week']  # Day of the week offset
    time_of_day = timedelta(hours=row['hours'], minutes=row['minutes'])  # Time of day as timedelta
    
    # Construct the datetime object
    x_datetime = base_date + timedelta(days=day_offset) + time_of_day
    
    # Add a scatter trace for each row
    trace = go.Scatter(
        x=[x_datetime],
        y=[row['Energy']],
        mode='markers',
        marker=dict(size=8),
        name=f"Day {int(row['Day of Week'])} Time {row['hours']}:{row['minutes']}"
    )
    traces.append(trace)

# Create the figure and add traces
fig = go.Figure(data=traces)

# Update layout
fig.update_layout(
    title='Energy Use by Time and Day of Week',
    xaxis_title='Date and Time',
    yaxis_title='Energy',
    template='plotly_dark'
)

# Show the plot
fig.show()


In [47]:
figure = {'data': [{'x': [0, 1], 'y': [0, 1]}],
          'layout': {'xaxis': {'range': [0, 5], 'autorange': False},
                     'yaxis': {'range': [0, 5], 'autorange': False},
                     'title': 'Start Title',
                     'updatemenus': [{'type': 'buttons',
                                      'buttons': [{'label': 'Play',
                                                   'method': 'animate',
                                                   'args': [None]}]}]
                    },
          'frames': [{'data': [{'x': [1, 2], 'y': [1, 2]}]},
                     {'data': [{'x': [1, 4], 'y': [1, 4]}]},
                     {'data': [{'x': [3, 4], 'y': [3, 4]}],
                      'layout': {'title': 'End Title'}}]}

iplot(figure)